In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Making a Dense Net CNN with Tensorflow to make Classification of Yoga Poses
I will be making a Multi Class Image Classification Problem

In [ ]:
# Initially we will build some Binary Classification for each Asanas
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import re
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

End to End Model Building with Input Layer -->  Convolution Layer --> Hidden Activation --> Pooling Layer --> Connected Layer --> Output Activation and Layer
With multiple Pooling and Convolution Layers

In [ ]:
# EDA
!ls ../input/yoga-pose-image-classification-dataset/dataset

In [ ]:
# Finding out the Number of Asanas
for dirpath,dirnames,filenames in os.walk("../input/yoga-pose-image-classification-dataset/dataset"):
    print(f"There are {len(dirnames)} asanas in the '{dirpath}'")
    

In [ ]:
# EDA  for any Asana
anantasana=len(os.listdir("../input/yoga-pose-image-classification-dataset/dataset/anantasana"))
anantasana
# 43 Images so we wont be splitting it to Train Test and Rather Make Predictions of Custom Images


In [ ]:
import pathlib
data_dir=pathlib.Path("../input/yoga-pose-image-classification-dataset/dataset")
class_names=np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names
# Creating an array of Classes for Predictions


In [ ]:
train_dir="../input/yoga-pose-image-classification-dataset/dataset"


In [ ]:
# Data Pre Processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Reescaling the Data
train_datagen=  ImageDataGenerator(rescale=1/255.)

In [ ]:
# Loading the Data from the dirs and turnit into Batches
train_data=train_datagen.flow_from_directory(train_dir,target_size=(224,224),batch_size=32,class_mode="categorical")

In [ ]:
# Importing the Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation

In [ ]:
# Creating a CNN Model for Training 
# Creating the Model
model=Sequential([Conv2D(10,3,input_shape=(224,224,3)),
                  Activation(activation="relu"),
                  Conv2D(10,3,activation="relu"),
                  MaxPool2D(),
                  Conv2D(10,3,activation="relu"),
                  Conv2D(10,3,activation="relu"),
                  MaxPool2D(),
                  Flatten(),
                  Dense(107,activation='softmax')
                 ])
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [ ]:
# Over Fitting the Model to the Dataset
history=model.fit(train_data,epochs=10,steps_per_epoch=len(train_data))

In [ ]:
# Evaluating the Model
testdata="../input/testdata1"
from PIL import Image
test_data=train_datagen.flow_from_directory(testdata,target_size=(224,224),class_mode="categorical")
# np_img = np_img[np.newaxis,:,:]


In [ ]:
pred=model.predict(test_data)

In [ ]:
import matplotlib.image as mpimg
# Create a function to import and image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
 
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode the read file into a tensor
  img = tf.image.decode_image(img)
  # Resize the image
  img = tf.image.resize(img, size=[img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
def pred_and_plot(model, test_img, class_names=class_names):
 
  # Import the target image and preprocess it
#   img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(test_img, axis=0))

  # Add in logic for multi-class & get pred_class name
  if len(pred[0]) > 1:
    pred_class = class_names[tf.argmax(pred[0])]
  else:
    pred_class = class_names[int(tf.round(pred[0]))]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);


In [ ]:
pred_and_plot(model=model,
              test_img=test_data,
              class_names=class_names)